In [1]:
import numpy as np
import pandas as pd
import os
import warnings
import librosa
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from random import shuffle
import scipy.stats as st

from pynwb import NWBHDF5IO
from pynwb.image import ImageSeries
from pynwb.epoch import TimeIntervals

In [2]:
# ----- User defined parameters -----
nwbpath = 'M:\\scn2a-paper-GWJSNH\\NWB-files'
usefield = 'annotated_behavior'
writeToDisk = True

In [3]:
# ----- File handling -----
allnwb = []
allnwb += [os.path.join(nwbpath,each) for each in os.listdir(nwbpath) if each.endswith('.nwb')]

In [7]:
allnwb
test = [allnwb[2]]

In [4]:
# ----- Get all times, durations of requested behavior -----

extra_starts = []
start_times = []
next_times = []

# loop over files
for i,fname in enumerate(allnwb):
    # open file
    io = NWBHDF5IO(os.path.join(nwbpath,fname), mode="r+") # <-- open in write to append snippets?
    nwbfile = io.read()
    
    pathsplit = fname.split('\\')
    recstem = pathsplit[-1].strip('.nwb')
    
    print(recstem)
       
    # get aggregated annotation table
    statsdf = nwbfile.intervals[usefield].to_dataframe()
    
    # find start time of assay
    assayStart = float(statsdf.start_time[statsdf.behavior=='Start assay'].to_list()[0]) # handle case where someone accidentally scored a Start twice
    if len(statsdf.start_time[statsdf.behavior=='Start assay'].to_list())>1:
        print(f'Extra start assay codes found in file {fname}.')
        extra_starts.append(fname)
        start_times.append(assayStart)
        nt = statsdf.start_time[[statsdf.start_time[statsdf.behavior=='Start assay'].index.tolist()][0][0]+1]
        next_times.append(nt)
    
    # get a list of all behaviors
    allbehav = np.unique(statsdf.behavior)
    
    # remove the start assay code
    behav = []
    behav += [each for each in allbehav if each != 'Start assay']
       
    # loop over each behavior to make a separated table
    for bhv in behav:
        print(f'Isolating behavior {bhv}')
        thisb = statsdf[statsdf.behavior==bhv]

        # make a table of start_time, stop_time, duration labeled with the behavior annotation
        thisb.start_time = thisb.start_time-assayStart
        thisb.stop_time = thisb.stop_time-assayStart

        # save that back to the NWB file                
        
        this_behav_table = TimeIntervals.from_dataframe(thisb,f'single_behavior_{bhv}',
                                          table_description=f'Assay-aligned annotations for behavior coded {bhv}.')

        # add data
        nwbfile.add_time_intervals(this_behav_table)
        
    # Add PPT data labeled by partner and chamber stranger        
    if nwbfile.lab_meta_data['vole_metadata'].assay_type == 'PPT':
        chamberopts = ['Left','Right']
        pchamb = nwbfile.lab_meta_data['vole_metadata'].assay_type__partner_chamber
        chamberopts.remove(pchamb)
        schamb = chamberopts[0]
        
        for bhv in behav:
            if pchamb in bhv:
                print(f'Isolating PPT behavior partner {bhv}')

                # make a table of start_time, stop_time, duration labeled with the behavior annotation        
                thisb = statsdf[statsdf.behavior==bhv]
                thisb.start_time = thisb.start_time-assayStart
                thisb.stop_time = thisb.stop_time-assayStart

                # rename everything for partner versus stranger
                # regenerate label
                label_components = bhv.split()
                unblind_label = bhv.replace(pchamb,'Partner')
                print(unblind_label)
                # replace label in dataframe
                thisb = thisb.replace(bhv,unblind_label)
                # save that back to the NWB file            
                this_behav_table = TimeIntervals.from_dataframe(thisb,f'single_behavior_{unblind_label}',
                                                  table_description=f'Assay-aligned annotations for behavior coded {bhv}.')

                # add data
                nwbfile.add_time_intervals(this_behav_table)
                
            elif schamb in bhv:
                print(f'Isolating PPT behavior stranger {bhv}')

                # make a table of start_time, stop_time, duration labeled with the behavior annotation        
                thisb = statsdf[statsdf.behavior==bhv]
                thisb.start_time = thisb.start_time-assayStart
                thisb.stop_time = thisb.stop_time-assayStart

                # rename everything for partner versus stranger
                # regenerate label
                label_components = bhv.split()
                unblind_label = bhv.replace(schamb,'Stranger')
                print(unblind_label)
                # replace label in dataframe
                thisb = thisb.replace(bhv,unblind_label)
                # save that back to the NWB file            
                this_behav_table = TimeIntervals.from_dataframe(thisb,f'single_behavior_{unblind_label}',
                                                  table_description=f'Assay-aligned annotations for behavior coded {bhv}.')

                # add data
                nwbfile.add_time_intervals(this_behav_table)

                
        
    if writeToDisk:    
        io.write(nwbfile)
    
    io.close()

Nov22_Pair15_aggressio
Isolating behavior Aggression receipt
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Isolating behavior Tussle


C:\Users\Nerissa\AppData\Local\Temp\ipykernel_6504\3665112809.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thisb.start_time = thisb.start_time-assayStart
C:\Users\Nerissa\AppData\Local\Temp\ipykernel_6504\3665112809.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thisb.stop_time = thisb.stop_time-assayStart


Nov22_Pair15_introductio
Isolating behavior Aggression receipt
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior Mount
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Isolating behavior Tussle
Nov22_Pair15_PPT
Isolating behavior Attack Right
Isolating behavior Center
Isolating behavior Huddle Left
Isolating behavior Huddle Right
Isolating behavior Interact Left
Isolating behavior Interact Right
Isolating behavior Left
Isolating behavior Right
Isolating PPT behavior partner Attack Right
Attack Partner
Isolating PPT behavior stranger Huddle Left
Huddle Stranger
Isolating PPT behavior partner Huddle Right
Huddle Partner
Isolating PPT behavior stranger Interact Left
Interact Stranger
Isolating PPT behavior partner Interact Right
Interact Partner
Isolating PPT behavior stranger Left
Stranger
Isolating PPT behavior partner Right
Partner


C:\Users\Nerissa\AppData\Local\Temp\ipykernel_6504\3665112809.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thisb.start_time = thisb.start_time-assayStart
C:\Users\Nerissa\AppData\Local\Temp\ipykernel_6504\3665112809.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thisb.stop_time = thisb.stop_time-assayStart
C:\Users\Nerissa\AppData\Local\Temp\ipykernel_6504\3665112809.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

Nov22_Pair15_reunio
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Nov22_Pair15_timedMating
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Nov22_Pair16_aggressio
Isolating behavior Aggression receipt
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Isolating behavior Tussle
Nov22_Pair16_introductio
Extra start assay codes found in file M:\scn2a-paper-GWJSNH\NWB-files\Nov22_Pair16_introduction.nwb.
Isolating behavior Aggression receipt
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Isolating be

...

Nov22_Pair42_introductio
Isolating behavior Aggression receipt
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Isolating behavior Tussle
Nov22_Pair42_PPT
Isolating behavior Center
Isolating behavior Huddle Left
Isolating behavior Huddle Right
Isolating behavior Interact Left
Isolating behavior Interact Right
Isolating behavior Left
Isolating behavior Right
Isolating PPT behavior partner Huddle Left
Huddle Partner
Isolating PPT behavior stranger Huddle Right
Huddle Stranger
Isolating PPT behavior partner Interact Left
Interact Partner
Isolating PPT behavior stranger Interact Right
Interact Stranger
Isolating PPT behavior partner Left
Partner
Isolating PPT behavior stranger Right
Stranger
Nov22_Pair42_timedMating
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Intromission
Isolating behavior Investigate
Isolating behavi

Nov22_Pair48_timedMating
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior Mount
Isolating behavior No interaction
Isolating behavior Sniff
Nov22_Pair49_aggressio
Extra start assay codes found in file M:\scn2a-paper-GWJSNH\NWB-files\Nov22_Pair49_aggression.nwb.
Isolating behavior Aggression receipt
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Isolating behavior Tussle
Nov22_Pair49_introductio
Isolating behavior Aggression receipt
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Nov22_Pair49_PPT
Isolating behavior Attack Left
Isolating behavior Center
Isolating behavior Huddle Left
Isolating behavior Huddle Right
Isolating behavior Inter

Nov22_Pair56_PPT
Isolating behavior Center
Isolating behavior Huddle Left
Isolating behavior Huddle Right
Isolating behavior Interact Left
Isolating behavior Interact Right
Isolating behavior Left
Isolating behavior Right
Isolating PPT behavior partner Huddle Left
Huddle Partner
Isolating PPT behavior stranger Huddle Right
Huddle Stranger
Isolating PPT behavior partner Interact Left
Interact Partner
Isolating PPT behavior stranger Interact Right
Interact Stranger
Isolating PPT behavior partner Left
Partner
Isolating PPT behavior stranger Right
Stranger
Nov22_Pair56_reunio
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Nov22_Pair56_timedMating
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Nov22_Pair57_aggressio
Isolating behavior Aggression receipt
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Investigate
Is

Nov22_Pair61_reunio
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Nov22_Pair61_timedMating
Isolating behavior Huddle
Isolating behavior Intromission
Isolating behavior Investigate
Isolating behavior Mount
Isolating behavior No interaction
Isolating behavior Sniff
Nov22_Pair62_aggressio
Isolating behavior Aggression receipt
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Isolating behavior Tussle
Nov22_Pair62_introductio
Isolating behavior Aggression receipt
Isolating behavior Defensive rear
Isolating behavior Defensive strike
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Nov22_Pair62_PPT
Isolating behavior Center
Isolating behavior Huddle Left
Isolating behavior

Nov22_Pair8_reunio
Isolating behavior Huddle
Isolating behavior Investigate
Isolating behavior No interaction
Isolating behavior Sniff
Nov22_Pair8_timedMating
Isolating behavior Ejaculate
Isolating behavior Huddle
Isolating behavior Intromission
Isolating behavior Investigate
Isolating behavior Mount
Isolating behavior No interaction
Isolating behavior Sniff
Isolating behavior Strike
Isolating behavior Tussle


In [48]:
print(start_times)
print(next_times)

[46.901, 51.153, 69.153, 49.874, 36.658, 78.157, 50.907, 54.921, 81.907, 3908.711, 3793.185, 89.792, 148.543, 127.12, 4037.046, 4047.874, 18.657, 77.043, 108.878, 69.707, 77.876, 3708.374, 3671.113, 134.121, 3679.213, 3.903, 3727.377, 25.455, 261.022]
[46.902, 51.154, 69.154, 49.875, 36.659, 78.158, 50.908, 54.922, 81.908, 3908.712, 3793.186, 89.793, 148.544, 127.121, 4037.047, 4047.875, 18.658, 77.044, 108.879, 69.708, 77.877, 3708.375, 3671.114, 134.122, 3679.214, 3.904, 3727.378, 25.456, 261.023]


In [44]:
nt

46.902

In [45]:
statsdf.head()

,start_time,stop_time,behavior,duration,atype
id,,,,,
0,46.901,46.901,Start assay,NaN,POINT
1,46.902,49.408,No interaction,2.506,STATE
2,49.409,50.656,Investigate,1.247,STATE
3,50.657,51.659,No interaction,1.002,STATE
4,51.660,53.409,Sniff,1.749,STATE


In [19]:
pinteract = nwbfile.intervals['single_behavior_Interact Partner'].to_dataframe()

In [20]:
pinteract

,start_time,stop_time,behavior,duration,atype
id,,,,,
3,13.001,14.499,Interact Partner,1.498,STATE
6,54.499,54.499,Interact Partner,0.001,STATE
10,81.000,81.495,Interact Partner,0.495,STATE
38,238.749,240.249,Interact Partner,1.500,STATE
40,242.001,244.002,Interact Partner,2.001,STATE
...,...,...,...,...,...
1794,10495.462,10496.465,Interact Partner,1.003,STATE
1797,10510.466,10511.966,Interact Partner,1.500,STATE
1815,10540.713,10541.965,Interact Partner,1.252,STATE


In [13]:
allbehav

array(['Attack Right', 'Center', 'Huddle Left', 'Huddle Right',
       'Interact Left', 'Interact Right', 'Left', 'Right', 'Start assay'],
      dtype=object)

In [ ]:
# ----- Notes -----
# when I want something like counts and latencies, I can just ask for the size and first instance from those tables
# they will be easy to feed into rasters and histograms